In [15]:
import uproot
import h5py
import numpy as np
import os
import awkward as ak

In [16]:
# pick random file
dir_root_lightjet = '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2/mc20_Run2/Ntuples/PeriodE/ZJets_MG/user.ltoffoli.mc20_13TeV.506193.MGPy8EG_3jets_HT2bias_BFilter.deriv.DAOD_PHYS.e8382_s3681_r13145_p5631.toffoGN2_v1_output.root'
fname = os.listdir(dir_root_lightjet)[3]

In [17]:
fname

'user.ltoffoli.35094310._000002.output.root'

In [18]:
tree = uproot.open(os.path.join(dir_root_lightjet,fname)+':nominal')

In [19]:
tree

<TTree 'nominal' (119 branches) at 0x7f769e6b9c50>

In [68]:
samples = tree.arrays(['el_e','mu_e','el_charge','mu_charge','el_pt','mu_pt', 'el_phi','mu_phi',\
                      'el_eta', 'mu_eta', 'jet_e'])[:3]

In [70]:
dir(samples)

['Mask',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__self_class__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__thisclass__',
 '_behavior',
 '_caches',
 '_layout',
 '_numbaview',
 'behavior',
 'caches',
 'el_charge',
 'el_e',
 'el_eta',
 'el_phi',
 'el_pt',
 'fields',
 'jet_e',
 'layout',
 'mask',
 'mu_charge',
 'mu_e',
 'mu_eta',
 'mu_phi',
 'mu_pt',
 'nbytes',
 'ndim',
 'numba_type',
 'slot0',
 'slot1',
 'slot2',
 'slot3',
 'slot4',
 'slot5',
 'slot6',
 'slot7',
 'slot8',
 'slot9',
 'to_list',
 'to_numpy',
 'tolist',
 'type']

In [73]:
samples = tree.arrays(['el_e','el_charge','el_pt', 'el_phi','el_eta'])[:3]

In [74]:
samples.layout

<RecordArray length="3">
    <field index="0" key="el_e">
        <ListOffsetArray64>
            <offsets><Index64 i="[0 2 4 6]" offset="0" length="4" at="0x00002c16ca90"/></offsets>
            <content><NumpyArray format="f" shape="3189056" data="203773 51661.3 315710 235376 185047 ... 44652.5 251212 204755 141797 78415.8" at="0x00002cd96fa0"/></content>
        </ListOffsetArray64>
    </field>
    <field index="1" key="el_charge">
        <ListOffsetArray64>
            <offsets><Index64 i="[0 2 4 6]" offset="0" length="4" at="0x000025299010"/></offsets>
            <content><NumpyArray format="f" shape="3189056" data="-1 1 1 -1 1 ... 1 -1 1 -1 1" at="0x000025ec3520"/></content>
        </ListOffsetArray64>
    </field>
    <field index="2" key="el_pt">
        <ListOffsetArray64>
            <offsets><Index64 i="[0 2 4 6]" offset="0" length="4" at="0x000026aeda30"/></offsets>
            <content><NumpyArray format="f" shape="3189056" data="178379 47609.6 263095 162134 124998 ...

In [75]:
samples

<Array [{el_e: [2.04e+05, ... 0.115]}] type='3 * {"el_e": var * float32, "el_cha...'>

In [76]:
samples[2].el_phi

<Array [1.25, 0.2] type='2 * float32'>

In [91]:
# sum over both electrons
ee_px = ak.sum(samples.el_pt * np.cos(samples.el_phi),axis=1)
ee_py = ak.sum(samples.el_pt * np.sin(samples.el_phi),axis=1)
ee_pz = ak.sum(samples.el_pt * np.sinh(samples.el_eta),axis=1)

In [92]:
ee_px

<Array [1.74e+05, 4.15e+05, 6.47e+04] type='3 * float32'>

In [95]:
ee_pt3d = ee_px**2 + ee_py**2 + ee_pz**2
ee_e3d = ak.sum(np.sqrt((samples.el_pt*np.cosh(samples.el_eta))**2 + (511e-3)**2),axis=1)

In [96]:
ee_e3d

<Array [2.55e+05, 5.51e+05, 2.11e+05] type='3 * float32'>

In [97]:
ee_e3d**2 - (ee_pt3d+ee_e3d**2)

<Array [-5.77e+10, -2.96e+11, -3.73e+10] type='3 * float32'>

In [40]:
samples['ee_m'] = ak.sum(np.sqrt(ee_e3d**2 - el_pt3d**2), axis=1)

In [41]:
np.sqrt(el_e3d**2 - el_pt3d**2)

<Array [[nan, 0], [0, 128], [64, nan]] type='3 * var * float32'>

In [42]:
el_pt3d

<Array [[2.04e+05, 5.17e+04, ... 2.59e+04]] type='3 * var * float32'>

In [43]:
el_e3d

<Array [[2.04e+05, 5.17e+04, ... 2.59e+04]] type='3 * var * float32'>

In [44]:
el_e3d**2 - el_pt3d**2

<Array [[-8.19e+03, 0], ... [4.1e+03, -64]] type='3 * var * float32'>